# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [188]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [189]:
#stocks=pd.read_csv('sp_500_stocks.csv')
#stocks

import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page containing the list of S&P 500 companies
wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Fetch the data from the URL
response = requests.get(wiki_url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the list of S&P 500 companies
table = soup.find('table', {'id': 'constituents'})

# Read the table into a pandas DataFrame
df = pd.read_html(str(table))[0]

# Extract the 'Symbol' column
sp500_tickers = df['Symbol']

# Display the first few ticker symbols

# Display the DataFrame
sp500_tickers
stocks= pd.DataFrame(sp500_tickers)
stocks


/var/folders/0r/w0c5h8sd4cx7n5gxq_7hsjhr0000gp/T/ipykernel_76629/3342166712.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
497,XYL
498,YUM
499,ZBRA
500,ZBH


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [190]:

import yfinance as yf
def get_pe_ratio(stock_symbol):
    
    
    stock = yf.Ticker(stock_symbol)

    if stock is None:
        return None,None
    
    # Fetch the stock info
    stock_info = stock.info
    
    # Extract the trailing P/E ratio and current price
    pe_ratio = stock_info.get('trailingPE')
    hist = stock.history(period="1d")
    current_price = hist['Close'].iloc[-1] if not hist.empty else None
    
    return pe_ratio, current_price

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [196]:
my_columns=['ticker', 'price', 'PE_ratio', 'sharesToBuy']
df= pd.DataFrame(columns=my_columns)
df
# Example: Get the P/E ratio for Apple (AAPL)

for stock in stocks['Symbol'][:20]:
    tickerSymbol= stock
    pe_ratio, current_price= get_pe_ratio(tickerSymbol)
    if pe_ratio is None or current_price is None:
        continue
    new_row={'price':current_price,'ticker':tickerSymbol ,'PE_ratio':pe_ratio,'sharesToBuy':np.nan}
    df = df._append(new_row, ignore_index = True)
df

/var/folders/0r/w0c5h8sd4cx7n5gxq_7hsjhr0000gp/T/ipykernel_76629/3747113781.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(new_row, ignore_index = True)


,ticker,price,PE_ratio,sharesToBuy
0,AOS,85.144997,22.115583,NaN
1,ABT,103.750000,32.421875,NaN
2,ABBV,168.500000,50.148810,NaN
3,ACN,296.334991,27.112076,NaN
4,ADBE,570.609985,51.313847,NaN
5,AMD,183.270004,261.907170,NaN
6,AES,18.440001,25.260275,NaN
7,AFL,91.129997,10.025302,NaN
8,A,129.919998,30.786732,NaN
9,APD,259.399994,23.432701,NaN


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [195]:
# Apply the mask to filter the DataFrame
filtered_df= filtered_df[filtered_df['PE_ratio']>0]
filtered_df= filtered_df.sort_values(by='PE_ratio', ascending= True)
print(filtered_df)
resulted_df= filtered_df[:50]
resulted_df.reset_index(inplace=True)
resulted_df.drop('index', axis=1, inplace=True)
resulted_df

   ticker       price    PE_ratio sharesToBuy
0     AOS   85.144997   22.115583         NaN
1     ABT  103.699997   32.406250         NaN
2    ABBV  168.570007   50.169647         NaN
3     ACN  296.214996   27.101097         NaN
4    ADBE  570.369995   51.292267         NaN
5     AMD  183.654999  262.314270         NaN
6     AES   18.445000   25.260275         NaN
7     AFL   91.080002   10.019802         NaN
8       A  129.919998   30.786732         NaN
9     APD  259.005005   23.397020         NaN
10   ABNB  150.630005   20.175370         NaN
11   AKAM   94.669998   23.618452         NaN
12    ALB   98.824997   35.817030         NaN
13    ARE  120.889999  114.056610         NaN
14   ALGN  254.240005   42.105785         NaN
15   ALLE  119.300003   19.461664         NaN
16    LNT   52.549999   19.109090         NaN
17    ALL  159.179993   34.781662         NaN
18  GOOGL  189.845001   29.116564         NaN
    ticker       price   PE_ratio sharesToBuy
37     APA   28.600000   3.209877 

/var/folders/0r/w0c5h8sd4cx7n5gxq_7hsjhr0000gp/T/ipykernel_76629/797367573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resulted_df.drop('index', axis=1, inplace=True)


,ticker,price,PE_ratio,sharesToBuy
0,APA,28.600000,3.209877,NaN
1,DAL,46.020000,5.900000,NaN
2,APTV,70.180000,6.595864,NaN
3,CE,132.179993,7.246710,NaN
4,CINF,115.699997,7.703062,NaN
5,DVN,47.029999,8.958095,NaN
6,DHI,135.559998,9.253243,NaN
7,CHTR,301.790009,9.766667,NaN
8,AFL,89.050003,9.796480,NaN
9,CMA,49.930000,9.926441,NaN


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio: 10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [27]:
position_size= float(portfolio_size)/ len(resulted_df.index)
position_size

for row in resulted_df.index:
    resulted_df.loc[row,'sharesToBuy'] = math.floor(position_size/ resulted_df.loc[row,'price'])

resulted_df

,ticker,price,PE_ratio,sharesToBuy
0,APA,28.600000,3.209877,6993
1,DAL,46.020000,5.900000,4345
2,APTV,70.180000,6.595864,2849
3,CE,132.179993,7.246710,1513
4,CINF,115.699997,7.703062,1728
5,DVN,47.029999,8.958095,4252
6,DHI,135.559998,9.253243,1475
7,CHTR,301.790009,9.766667,662
8,AFL,89.050003,9.796480,2245
9,CMA,49.930000,9.926441,4005


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [207]:
import yfinance as yf
def get_ratios(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    if stock is None:
        return None,None,None,None,None,None
    
    # Fetch the stock info
    stock_info = stock.info
    
    hist = stock.history(period="1d")
    try:
        current_price = stock.history(period="1d")['Close'].iloc[-1]
    except:
        current_price= None
    # Extract the trailing P/E ratio and current price
    pe_ratio = stock_info.get('trailingPE')
    # Get the Price to Book ratio
    pb_ratio = stock_info.get('priceToBook')
     # Get the Price to Sales ratio
    ps_ratio = stock_info.get('priceToSalesTrailing12Months')
    # Get the EV/EBITDA ratio
    ev_ebitda_ratio = stock_info.get('enterpriseToEbitda')
    if ev_ebitda_ratio is None:
        ev_ebitda_ratio= np.NaN
        
    # Get the Enterprise Value (EV)
    enterprise_value = stock_info.get('enterpriseValue')
    # Get the Gross Profit
    gross_profit = stock_info.get('grossProfits')
    # Calculate the EV/GP ratio

    try:
        ev_gp_ratio = enterprise_value / gross_profit
    except:
        ev_gp_ratio= np.NaN

    return current_price,pe_ratio, pb_ratio, ps_ratio, ev_ebitda_ratio, ev_gp_ratio

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [208]:
rv_columns=[
    'Ticker',
    'Price',
    'SharesToBuy',
    'pe',
    'pe_percentile',
    'pb',
    'pb_percentile',
    'ps',
    'ps_percentile',
    'ev_ebitda',
    'ev_ebitda_percentile',
    'ev_gp',
    'ev_gp_percentile',
    'rv_score'
    
]
df= pd.DataFrame(columns=rv_columns)
df

for stock in stocks['Symbol'][:100]:
    tickerSymbol= stock
    current_price, pe_ratio, pb_ratio, ps_ratio, ev_ebitda_ratio, ev_gp_ratio  = get_ratios(tickerSymbol)
    if pe_ratio is None or current_price is None or pb_ratio is None or ps_ratio is None or ev_ebitda_ratio is None :
         continue
    
    new_row={'Price':current_price,'Ticker':tickerSymbol ,'pe':pe_ratio,'pb': pb_ratio,'ps':ps_ratio,'ev_ebitda':ev_ebitda_ratio,'ev_gp': ev_gp_ratio, 'SharesToBuy':np.nan}
    df = df._append(new_row, ignore_index = True)
df

/var/folders/0r/w0c5h8sd4cx7n5gxq_7hsjhr0000gp/T/ipykernel_76629/189567895.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append(new_row, ignore_index = True)
BRK.B: No data found, symbol may be delisted
BRK.B: No data found, symbol may be delisted
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']


,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,AOS,85.760002,NaN,22.275326,NaN,6.684334,NaN,3.254274,NaN,14.562,NaN,NaN,NaN,NaN
1,ABT,103.680000,NaN,32.399998,NaN,4.647452,NaN,4.472669,NaN,18.090,NaN,NaN,NaN,NaN
2,ABBV,169.240707,NaN,50.369260,NaN,37.318790,NaN,5.493393,NaN,13.526,NaN,NaN,NaN,NaN
3,ACN,300.579987,NaN,27.500456,NaN,6.788779,NaN,2.920140,NaN,16.401,NaN,NaN,NaN,NaN
4,ADBE,562.820007,NaN,50.613310,NaN,17.025229,NaN,12.215692,NaN,31.661,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,CE,139.320007,NaN,7.638158,NaN,2.151760,NaN,1.422371,NaN,15.116,NaN,NaN,NaN,NaN
81,COR,220.330002,NaN,24.106127,NaN,40.561485,NaN,0.156903,NaN,12.466,NaN,NaN,NaN,NaN
82,CNC,67.519997,NaN,13.317554,NaN,1.343441,NaN,0.252328,NaN,5.546,NaN,NaN,NaN,NaN
83,CNP,29.885000,NaN,21.045776,NaN,1.892654,NaN,2.239446,NaN,12.106,NaN,NaN,NaN,NaN


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [219]:
# Custom function to check for None
def is_none(val):
    return val is None

# Apply the custom function to check for None values
none_mask = df.applymap(is_none)

# Check if any cell has None value
has_none = none_mask.any().any()

filtered_df= df
filtered_df= filtered_df[filtered_df['pe']>0]
filtered_df= filtered_df[filtered_df['pb']>0]
filtered_df= filtered_df[filtered_df['ps']>0]
filtered_df= filtered_df[filtered_df['ev_ebitda']>0]
filtered_df= filtered_df[filtered_df['ev_ebitda'] != np.NaN]
resulted_df= filtered_df
resulted_df.reset_index(inplace=True)
resulted_df.drop('index', axis=1, inplace=True)
df=resulted_df
df


/var/folders/0r/w0c5h8sd4cx7n5gxq_7hsjhr0000gp/T/ipykernel_76629/2629362442.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  none_mask = df.applymap(is_none)


,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,AOS,85.724998,NaN,22.266233,36.0,6.681605,62.0,3.252945,40.0,14.562,NaN,NaN,NaN,NaN
1,ABT,103.760002,NaN,32.425000,64.0,4.651038,48.0,4.476120,50.0,18.090,NaN,NaN,NaN,NaN
2,ABBV,170.029999,NaN,50.604168,82.0,37.492836,98.0,5.519013,58.0,13.526,NaN,NaN,NaN,NaN
3,ACN,301.109985,NaN,27.548946,46.0,6.800749,64.0,2.925289,34.0,16.401,NaN,NaN,NaN,NaN
4,ADBE,563.890015,NaN,50.709534,84.0,17.057596,88.0,12.238916,92.0,31.661,NaN,NaN,NaN,NaN
5,AMD,182.350006,NaN,260.500000,100.0,5.250050,54.0,12.926936,94.0,76.701,NaN,NaN,NaN,NaN
6,AES,18.555000,NaN,25.417809,42.0,4.546680,46.0,1.053734,14.0,13.408,NaN,NaN,NaN,NaN
7,AFL,91.519997,NaN,10.068206,10.0,2.217430,26.0,2.689335,30.0,8.634,NaN,NaN,NaN,NaN
8,A,130.369995,NaN,30.893366,60.0,6.117497,60.0,5.771033,60.0,23.653,NaN,NaN,NaN,NaN
9,APD,261.630005,NaN,23.634148,38.0,3.899978,42.0,4.785886,52.0,16.998,NaN,NaN,NaN,NaN


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [220]:
metrics={
    'pe':'pe_percentile',
    'pb':'pb_percentile',
    'ps':'ps_percentile',
    'ev_ebitda':'ev_ebitda_percentile',
}

for metric in metrics.keys():
    for row in df.index:
        df.loc[row, metrics[metric]]= stats.percentileofscore( df[metric] , df.loc[row,metric])

df

,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,AOS,85.724998,NaN,22.266233,34.042553,6.681605,61.702128,3.252945,36.170213,14.562,36.170213,NaN,NaN,NaN
1,ABT,103.760002,NaN,32.425000,63.829787,4.651038,48.93617,4.476120,46.808511,18.090,53.191489,NaN,NaN,NaN
2,ABBV,170.029999,NaN,50.604168,80.851064,37.492836,97.87234,5.519013,55.319149,13.526,29.787234,NaN,NaN,NaN
3,ACN,301.109985,NaN,27.548946,44.680851,6.800749,63.829787,2.925289,31.914894,16.401,42.553191,NaN,NaN,NaN
4,ADBE,563.890015,NaN,50.709534,82.978723,17.057596,87.234043,12.238916,91.489362,31.661,87.234043,NaN,NaN,NaN
5,AMD,182.350006,NaN,260.500000,100.0,5.250050,55.319149,12.926936,93.617021,76.701,97.87234,NaN,NaN,NaN
6,AES,18.555000,NaN,25.417809,40.425532,4.546680,46.808511,1.053734,14.893617,13.408,27.659574,NaN,NaN,NaN
7,AFL,91.519997,NaN,10.068206,10.638298,2.217430,25.531915,2.689335,27.659574,8.634,10.638298,NaN,NaN,NaN
8,A,130.369995,NaN,30.893366,59.574468,6.117497,59.574468,5.771033,57.446809,23.653,76.595745,NaN,NaN,NaN
9,APD,261.630005,NaN,23.634148,36.170213,3.899978,42.553191,4.785886,48.93617,16.998,46.808511,NaN,NaN,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [228]:
from statistics import mean

for row in df.index:
    value_percentiles=[]
    for metric in metrics.keys():
        value_percentiles.append(df.loc[row, metrics[metric]])
    mean_val= mean(value_percentiles)
    
    df.loc[row,'rv_score']= mean_val

df

,level_0,index,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,0,23,AIG,76.389999,NaN,11.418535,14.893617,1.181502,4.255319,1.048186,12.765957,5.538,4.255319,NaN,NaN,9.042553
1,1,40,T,18.705000,NaN,10.056452,8.510638,1.282922,6.382979,1.096479,17.021277,7.092,6.382979,NaN,NaN,9.574468
2,2,36,ADM,65.165001,NaN,11.372601,12.765957,1.408882,8.510638,0.351325,2.12766,9.415,17.021277,NaN,NaN,10.106383
3,3,34,APTV,71.769997,NaN,6.745300,4.255319,1.755798,17.021277,0.969797,8.510638,8.730,12.765957,NaN,NaN,10.638298
4,4,39,AIZ,165.750000,NaN,11.590909,17.021277,1.752614,14.893617,0.757914,6.382979,7.127,8.510638,NaN,NaN,11.702128
5,5,31,APA,29.270000,NaN,3.285073,2.12766,3.380688,38.297872,1.344153,19.148936,3.424,2.12766,NaN,NaN,15.425532
6,6,35,ACGL,96.324997,NaN,7.602604,6.382979,1.943525,21.276596,2.510548,25.531915,9.077,14.893617,NaN,NaN,17.021277
7,7,7,AFL,91.519997,NaN,10.068206,10.638298,2.217430,25.531915,2.689335,27.659574,8.634,10.638298,NaN,NaN,18.617021
8,8,21,AEE,73.389999,NaN,16.832567,21.276596,1.709846,12.765957,2.788674,29.787234,11.364,19.148936,NaN,NaN,20.744681
9,9,22,AEP,89.980003,NaN,16.756054,19.148936,1.837789,19.148936,2.455360,23.404255,12.308,23.404255,NaN,NaN,21.276596


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [232]:
df.sort_values('rv_score' , ascending= True, inplace= True)
df.drop('level_0', axis=1, inplace= True)
df



,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,AIG,76.389999,NaN,11.418535,14.893617,1.181502,4.255319,1.048186,12.765957,5.538,4.255319,NaN,NaN,9.042553
1,T,18.705000,NaN,10.056452,8.510638,1.282922,6.382979,1.096479,17.021277,7.092,6.382979,NaN,NaN,9.574468
2,ADM,65.165001,NaN,11.372601,12.765957,1.408882,8.510638,0.351325,2.12766,9.415,17.021277,NaN,NaN,10.106383
3,APTV,71.769997,NaN,6.745300,4.255319,1.755798,17.021277,0.969797,8.510638,8.730,12.765957,NaN,NaN,10.638298
4,AIZ,165.750000,NaN,11.590909,17.021277,1.752614,14.893617,0.757914,6.382979,7.127,8.510638,NaN,NaN,11.702128
5,APA,29.270000,NaN,3.285073,2.12766,3.380688,38.297872,1.344153,19.148936,3.424,2.12766,NaN,NaN,15.425532
6,ACGL,96.324997,NaN,7.602604,6.382979,1.943525,21.276596,2.510548,25.531915,9.077,14.893617,NaN,NaN,17.021277
7,AFL,91.519997,NaN,10.068206,10.638298,2.217430,25.531915,2.689335,27.659574,8.634,10.638298,NaN,NaN,18.617021
8,AEE,73.389999,NaN,16.832567,21.276596,1.709846,12.765957,2.788674,29.787234,11.364,19.148936,NaN,NaN,20.744681
9,AEP,89.980003,NaN,16.756054,19.148936,1.837789,19.148936,2.455360,23.404255,12.308,23.404255,NaN,NaN,21.276596


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [237]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [239]:
portfolio_input()


Enter the value of your portfolio: 10000


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [241]:
position_size= float(portfolio_size)/ len(resulted_df.index)
position_size

for row in df.index:
    df.loc[row,'SharesToBuy'] = math.floor(position_size/ resulted_df.loc[row,'Price'])

resulted_df

,Ticker,Price,SharesToBuy,pe,pe_percentile,pb,pb_percentile,ps,ps_percentile,ev_ebitda,ev_ebitda_percentile,ev_gp,ev_gp_percentile,rv_score
0,AIG,76.389999,2,11.418535,14.893617,1.181502,4.255319,1.048186,12.765957,5.538,4.255319,NaN,NaN,9.042553
1,T,18.705000,11,10.056452,8.510638,1.282922,6.382979,1.096479,17.021277,7.092,6.382979,NaN,NaN,9.574468
2,ADM,65.165001,3,11.372601,12.765957,1.408882,8.510638,0.351325,2.12766,9.415,17.021277,NaN,NaN,10.106383
3,APTV,71.769997,2,6.745300,4.255319,1.755798,17.021277,0.969797,8.510638,8.730,12.765957,NaN,NaN,10.638298
4,AIZ,165.750000,1,11.590909,17.021277,1.752614,14.893617,0.757914,6.382979,7.127,8.510638,NaN,NaN,11.702128
5,APA,29.270000,7,3.285073,2.12766,3.380688,38.297872,1.344153,19.148936,3.424,2.12766,NaN,NaN,15.425532
6,ACGL,96.324997,2,7.602604,6.382979,1.943525,21.276596,2.510548,25.531915,9.077,14.893617,NaN,NaN,17.021277
7,AFL,91.519997,2,10.068206,10.638298,2.217430,25.531915,2.689335,27.659574,8.634,10.638298,NaN,NaN,18.617021
8,AEE,73.389999,2,16.832567,21.276596,1.709846,12.765957,2.788674,29.787234,11.364,19.148936,NaN,NaN,20.744681
9,AEP,89.980003,2,16.756054,19.148936,1.837789,19.148936,2.455360,23.404255,12.308,23.404255,NaN,NaN,21.276596


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [242]:
writer= pd.ExcelWriter('quantitative_value_strat.xlsx',engine='xlsxwriter')
df.to_excel(writer,sheet_name='To_buy_trades', index= False)
writer.close()

## Saving Our Excel Output
As before, saving our Excel output is very easy: